In [9]:
import csv
import pandas as pd
import numpy as np

import folium
import folium.plugins as plugins
import ipywidgets as widgets
from IPython.display import HTML

#parse and load csv file into a Pandas DataFrame
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Army_Group','Num_Name'])

#create a dictionary of the army groups and nums to use in a selection widget
army_groups = dict()
for group_keys in grouped_div.groups.keys():
    army_group = group_keys[0]
    div_num = group_keys[1]
    if isinstance(div_num,str):
        if army_group in army_groups:
            army_groups[army_group].append(div_num)
        else:
            army_groups[army_group] = [div_num]

#output widget for the map
out = widgets.Output()

#selection widgets for army group and num
group_sel = widgets.Dropdown(options=army_groups.keys())
div_sel = widgets.Dropdown(options=['-'])

#event handler
def on_group_sel(change):
    if change['new'] in army_groups:
        div_sel.options = army_groups[change['new']]

group_sel.observe(on_group_sel,'value')

def on_div_sel(change):
    if change['new'] is not None:
        plot_div_movement(group_sel.value,change['new'])
    
div_sel.observe(on_div_sel,'value')

#main function to plot the movement of a selected group and num
def plot_div_movement(army_group,num_name):
    division_map = folium.Map([54.78, 32.04],zoom_start=6)
    div_df = grouped_div.get_group((army_group,num_name))[['POINT_Y','POINT_X','OBJECTID','MAP_DATE']]
    features = [
      {
          "type": "Feature",
          "geometry": {
              "type": "Point",
              "coordinates": [row.POINT_X,row.POINT_Y]
          },
          "properties":{
              "icon": "marker", 
              "iconstyle": {
                  "iconSize": [30, 30], 
                  "iconUrl":"https://github.com/rkalyanapurdue/smolensk/raw/master/tank.png"}, 
              "time": str(row.MAP_DATE)[:10]}
      }
      for row in div_df.itertuples()
  ]

    plugins.TimestampedGeoJson({
      'type': 'FeatureCollection',
      'features': features
  }, add_last_point=True).add_to(division_map)

    out.clear_output()
    iframe = division_map._repr_html_()
    out.append_display_data(HTML(iframe))
    return

#output UI
sel_ui = widgets.HBox([group_sel,div_sel])
res = widgets.VBox([sel_ui,out])
res